## This code allows for FAST SYMBOLIC calculation of MONO and BI electronic terms between determinants built on NON-ORTHOGONAL MOs
It uses D4 symmetry and spin selection rules. It is developed for Hermitian matrices but can be updated consequently for non-Hermitian matrices.
Updated: Pablo ROSEIRO - 23.06.2023

In [1]:
# Last updated: 08-02-2022

import numpy as np
import sympy as sym
from sympy.core.function import _coeff_isneg as coeff_isneg

from sympy.physics.quantum import Bra

# Make symoblic elements out of (g,u) = AOs of active chromophore, (j,v) = AOs of environment chromophore, and (S1,S3) = overlaps 
g = sym.Symbol('g')
u = sym.Symbol('u')
j = sym.Symbol('j')
v = sym.Symbol('v')
S1 = sym.Symbol('S1')
S3 = sym.Symbol('S3')

In [2]:
# First case: doubly occupied Singlet environment = |j -j|
a1 = [g,-g,j,-j]
b1 = [u,-u,j,-j]
c1 = [g,-u,j,-j]
d1 = [u,-g,j,-j]

# Second case: doubly occupied excited Singlet environment = |v -v|
a2 = [g,-g,v,-v]
b2 = [u,-u,v,-v]
c2 = [g,-u,v,-v]
d2 = [u,-g,v,-v]

# Third and Fourth cases: open-shell excited environments = |j -v| and |v -j|
a3 = [g,-g,j,-v]
b3 = [u,-u,j,-v]
c3 = [g,-u,j,-v]
d3 = [u,-g,j,-v]

a4 = [g,-g,v,-j]
b4 = [u,-u,v,-j]
c4 = [g,-u,v,-j]
d4 = [u,-g,v,-j]

# These determinants are ms = -2,0,+2 with a ms=-1,+1 on active and environment
e1 = [g,u,j,v]
f1 = [-g,-u,j,v]

e2 = [g,u,-j,-v]
f2 = [-g,-u,-j,-v]

e3 = [g,u,j,-v]
f3 = [-g,-u,j,-v]

e4 = [g,u,v,-j]
f4 = [-g,-u,v,-j]

In [3]:
# This script was coded the 21.02.2022
# Detailed comments were added the 23.06.2023

def inter_det(A,B):
    """
    This script constructs the S overlap matrix between determinants A and B
    """
    det = []
    for i in range(len(A)):
        line = []
        for k in range(len(B)):
            if (A[i] * B[k]) in {-g*u,-u*g,-j*v,-v*j,-g*v, -v*g,-j*u,-u*j,-g*g,-u*u,-j*j,-v*v,-g*j,-j*g,-u*v,-v*u}:
                value = 0

            elif (A[i] * B[k]) in {g*u,u*g,j*v,v*j,g*v,v*g,j*u,u*j}:
                value = 0

            elif (A[i] * B[k]) in {g*g,u*u,j*j,v*v}:
                value = 1

            elif (A[i] * B[k]) in {g*j,j*g}:
                value = S1

            elif (A[i] * B[k]) in {u*v,v*u}:
                value = S3

            line.append(value)
        det.append(line)
    return det

def permutator_mono(expression):
    """
    This code uses D4 symmetry to switch a*a:b*b in monoelectronic integrals 
    (e.g. jj is seen as gg, gu is 0 due to symmetry).
    """
    if expression == 0:
        return expression
    else:
        new_expr = expression.subs({j*j:g*g, j*u:0, v*v:u*u, j*v:0, g*u:0, g*v:0})
        return new_expr

    
def permutator_bi(expression):
    """
    This code uses D4 symmetry to switch Bra(b*b)*Ket(a*a):Bra(a*a)*Ket(b*b) in bielectronic integrals
    (e.g. (jj,jj) = (gg,gg), (gu,gg) = 0 due to symmetry).
    """
    if expression == 0:
        return expression
    else:
        diction = {Bra(j*j)*Bra(j*j).dual:Bra(g*g)*Bra(g*g).dual,
                   Bra(v*v)*Bra(v*v).dual:Bra(u*u)*Bra(u*u).dual,
                   Bra(j*j)*Bra(g*g).dual:Bra(g*g)*Bra(j*j).dual,
                   Bra(v*v)*Bra(u*u).dual:Bra(u*u)*Bra(v*v).dual,
                   Bra(j*j)*Bra(v*v).dual:Bra(g*g)*Bra(u*u).dual,
                   Bra(v*v)*Bra(j*j).dual:Bra(g*g)*Bra(u*u).dual,
                   Bra(u*u)*Bra(g*g).dual:Bra(g*g)*Bra(u*u).dual,
                   Bra(j*v)*Bra(j*v).dual:Bra(g*u)*Bra(u*g).dual,
                   Bra(j*j)*Bra(u*u).dual:Bra(g*g)*Bra(v*v).dual,
                   Bra(u*u)*Bra(j*j).dual:Bra(g*g)*Bra(v*v).dual,
                   Bra(j*u)*Bra(j*u).dual:Bra(g*v)*Bra(g*v).dual,
                   Bra(j*v)*Bra(g*u).dual:Bra(g*u)*Bra(j*v).dual,
                   Bra(j*u)*Bra(g*v).dual:Bra(g*v)*Bra(j*u).dual,
                   Bra(u*v)*Bra(g*j).dual:Bra(g*j)*Bra(u*v).dual,
                   Bra(g*j)*Bra(g*g).dual:Bra(g*g)*Bra(g*j).dual,
                   Bra(g*j)*Bra(j*j).dual:Bra(g*g)*Bra(g*j).dual,
                   Bra(j*j)*Bra(g*j).dual:Bra(g*g)*Bra(g*j).dual,
                   Bra(u*u)*Bra(g*j).dual:Bra(g*j)*Bra(u*u).dual,
                   Bra(v*v)*Bra(g*j).dual:Bra(g*j)*Bra(u*u).dual,
                   Bra(g*j)*Bra(v*v).dual:Bra(g*j)*Bra(u*u).dual,
                   Bra(j*u)*Bra(g*u).dual:Bra(g*u)*Bra(u*j).dual,
                   Bra(g*v)*Bra(j*v).dual:Bra(g*u)*Bra(u*j).dual,
                   Bra(j*v)*Bra(g*v).dual:Bra(g*u)*Bra(u*j).dual,
                   Bra(u*v)*Bra(u*u).dual:Bra(u*u)*Bra(u*v).dual,
                   Bra(u*v)*Bra(v*v).dual:Bra(u*u)*Bra(u*v).dual,
                   Bra(v*v)*Bra(u*v).dual:Bra(u*u)*Bra(u*v).dual,
                   Bra(u*v)*Bra(g*g).dual:Bra(g*g)*Bra(u*v).dual,
                   Bra(u*v)*Bra(j*j).dual:Bra(g*g)*Bra(u*v).dual,
                   Bra(j*j)*Bra(u*v).dual:Bra(g*g)*Bra(u*v).dual,
                   Bra(v*g)*Bra(g*u).dual:Bra(g*u)*Bra(v*g).dual,
                   Bra(u*j)*Bra(j*v).dual:Bra(g*u)*Bra(v*g).dual,
                   Bra(j*v)*Bra(u*j).dual:Bra(g*u)*Bra(v*g).dual,
                   
                   Bra(g*u)*Bra(g*g).dual:0,
                   Bra(g*g)*Bra(g*u).dual:0,
                   Bra(g*u)*Bra(u*u).dual:0,
                   Bra(u*u)*Bra(g*u).dual:0,
                   Bra(g*u)*Bra(j*j).dual:0,
                   Bra(j*j)*Bra(g*u).dual:0,
                   Bra(g*u)*Bra(v*v).dual:0,
                   Bra(v*v)*Bra(g*u).dual:0,
                   Bra(g*u)*Bra(g*j).dual:0,
                   Bra(g*j)*Bra(g*u).dual:0,
                   Bra(g*u)*Bra(u*v).dual:0,
                   Bra(u*v)*Bra(g*u).dual:0,
                   Bra(g*v)*Bra(g*g).dual:0,
                   Bra(g*g)*Bra(g*v).dual:0,
                   Bra(g*v)*Bra(u*u).dual:0,
                   Bra(u*u)*Bra(g*v).dual:0,
                   Bra(g*v)*Bra(j*j).dual:0,
                   Bra(j*j)*Bra(g*v).dual:0,
                   Bra(g*v)*Bra(v*v).dual:0,
                   Bra(v*v)*Bra(g*v).dual:0,
                   Bra(g*v)*Bra(g*j).dual:0,
                   Bra(g*j)*Bra(g*v).dual:0,
                   Bra(g*v)*Bra(u*v).dual:0,
                   Bra(u*v)*Bra(g*v).dual:0,
                   Bra(j*v)*Bra(g*g).dual:0,
                   Bra(g*g)*Bra(j*v).dual:0,
                   Bra(j*v)*Bra(u*u).dual:0,
                   Bra(u*u)*Bra(j*v).dual:0,
                   Bra(j*v)*Bra(j*j).dual:0,
                   Bra(j*j)*Bra(j*v).dual:0,
                   Bra(j*v)*Bra(v*v).dual:0,
                   Bra(v*v)*Bra(j*v).dual:0,
                   Bra(j*v)*Bra(g*j).dual:0,
                   Bra(g*j)*Bra(j*v).dual:0,
                   Bra(j*v)*Bra(u*v).dual:0,
                   Bra(u*v)*Bra(j*v).dual:0,
                   Bra(j*u)*Bra(g*g).dual:0,
                   Bra(g*g)*Bra(j*u).dual:0,
                   Bra(j*u)*Bra(u*u).dual:0,
                   Bra(u*u)*Bra(j*u).dual:0,
                   Bra(j*u)*Bra(j*j).dual:0,
                   Bra(j*j)*Bra(j*u).dual:0,
                   Bra(j*u)*Bra(v*v).dual:0,
                   Bra(v*v)*Bra(j*u).dual:0,
                   Bra(j*u)*Bra(g*j).dual:0,
                   Bra(g*j)*Bra(j*u).dual:0,
                   Bra(j*u)*Bra(u*v).dual:0,
                   Bra(u*v)*Bra(j*u).dual:0}
        # Here the terms in expression are subsituted by the one defined in new_expr
        new_expr = expression.subs(diction)
  
    return new_expr

def minor_det_mono(A,B):
    """
    This code prepares the monoelectronic terms between A and B, corrected by their respectives minor determinants.
    """
    intero_det = inter_det(A,B)

    total = 0
    # THIS PART CREATES THE MINOR DET # i and k
    for i in range(len(intero_det)):
        for k in range(len(intero_det)):
            matrix_det = inter_det(A,B)
            for l in matrix_det:
                del l[k] # remove column k
            del matrix_det[i] # remove line i
            
            # THIS PART COMPUTES THE MINOR DET and MULTIPLIES IT BY THE CORRECT INTEGRAL
            M = sym.Matrix(matrix_det)

            
            if coeff_isneg(A[i] * B[k]):
                # This is a <a|h|-b> integral, automatically zero
                pass
            else:
                # This is a <a|h|b> integral
                if (i + k) %2 == 0:
                    # The number of permutations to bring i and k into the first row and column is even
                    total = total + M.det() * A[i] * B[k]
                else:
                    # The number of permutations is odd, one needs to add a -1 prefactor
                    total = total - M.det() * A[i] * B[k]
                    
    # The last step simplifies the expression by using D4 symmetry arguments
    return permutator_mono(total)


def minor_det_bi(A,B):
    """
    This code prepares the bielectronic terms between A and B, corrected by their respectives minor determinants.
    """
    intero_det = inter_det(A,B)
    total = 0
    # THIS PART CREATES THE MINOR DET # i=p, j=q, k=r, l=s          i < k and j < l
    for p in range(len(intero_det)): # row
        for q in range(len(intero_det)): # column
            for r in range(p+1, len(intero_det)): # row
                for s in range(q+1, len(intero_det)): # column

                    matrix_det = inter_det(A,B)
                    for l in matrix_det:
                        del l[q] # remove column q
                        del l[s-1] #remove colum s
                    del matrix_det[p] # remove row p
                    del matrix_det[r-1] # remove row r-1 (because p was removed and p<r)
                    #print(matrix_det)

                    # THIS PART COMPUTES THE MINOR DET and MULTIPLIES IT BY THE CORRECT INTEGRAL
                    M = sym.Matrix(matrix_det)
                    # See if the number of permutations is even or odd
                    sum_permut = p + q + r + s
                    
                    if (coeff_isneg(-1 * A[p] * B[q]) and coeff_isneg(-1 * A[r] * B[s])):
                        # p and q have the same spin-part, same for r and s.
                        if (coeff_isneg(A[p] * B[s]) or coeff_isneg(A[r] * B[q])):
                            # The (ps,rq) integral is zero due to spin selection rules: (p and s) or (r and q) do not have the same spin-part
                            if (sum_permut % 2 == 0):
                                total = total + M.det() * (Bra(A[p] * B[q])*Bra(A[r] * B[s]).dual)
                                    
                            else:
                                total = total - M.det() * (Bra(A[p] * B[q])*Bra(A[r] * B[s]).dual)
                                   
                        else:
                            # One need to consider the (ps,rq) integral
                            if (sum_permut % 2 == 0):
                                total = total + M.det() * (Bra(A[p] * B[q])*Bra(A[r] * B[s]).dual - Bra(A[p] * B[s])*Bra(A[r] * B[q]).dual)

                            else:
                                total = total - M.det() * (Bra(A[p] * B[q])*Bra(A[r] * B[s]).dual - Bra(A[p] * B[s])*Bra(A[r] * B[q]).dual)
                                   
                    elif (coeff_isneg(A[p] * B[q]) and coeff_isneg(A[r] * B[s])):
                        # p and q have the different spin-part, same for r and s.
                        # The (pq,rs) integral is zero by spin selection.
                        if (coeff_isneg(A[p] * B[s]) or coeff_isneg(A[r] * B[q])):
                            # The (ps,rq) integral is zero due to spin selection rules
                            # There is no integrals.
                            pass
                        else:
                            # The (ps,rq) integral is non-zero
                            if (sum_permut % 2 == 0):
                                total = total - M.det() * (Bra(A[p] * B[s])*Bra(A[r] * B[q]).dual)

                            else:
                                total = total + M.det() * (Bra(A[p] * B[s])*Bra(A[r] * B[q]).dual)
                    
                    else:
                        # There is 3 up and 1 down, or 1 up and 3 down, thus this is zero by spin selection rules.
                        pass
    
    # The last step simplifies the expression by using D4 symmetry arguments
    return permutator_bi(total)

def minor_det_total(A,B):
    """
    This code sums up the mono- and bi-electronic terms, corrected with their respective determinants.
    """
    # all the monoelectronic integrals
    mono = minor_det_mono(A, B)
    # all the bielectornic integrals
    bi = minor_det_bi(A,B)
    result = mono + bi
    return result


In [4]:
# This part of the code prepares the list of matrix elements from a list of determinants
#

## CREATE THE LIST OF DETERMINANTS
# list_det_name = ["a1", "b1", "c1", "d1", "e1", "f1", "a2", "b2",
#                  "c2", "d2", "e2", "f2", "a3", "b3", "c3", "d3",
#                  "e3", "f3", "a4", "b4", "c4", "d4", "e4", "f4"]

# list_det = [a1, b1, c1, d1, e1, f1, a2, b2,
#             c2, d2, e2, f2, a3, b3, c3, d3,
#             e3, f3, a4, b4, c4, d4, e4, f4]

# if len(list_det_name) - len(list_det) !=0:
#     print("Please update list_det")

# ## CREATE THE LIST OF MATRIX ELEMENTS
# list_name = []
# list_mat = []        
# for i in range(len(list_det_name)):
#     for j in range(i,len(list_det_name)):
#         list_name.append(list_det_name[i]+list_det_name[j])
#         list_mat.append([list_det[i],list_det[j]])
        
## THIS ALLOWS TO GET THE FOLLOWING LISTS
# Note: it is important to extract these list rather than start from the previously calculated ones, as they cause an issue with the python variable pointer
list_name_mat =['a1a1', 'a1b1', 'a1c1', 'a1d1', 'a1e1', 'a1f1','a1a2', 'a1b2', 'a1c2', 'a1d2', 'a1e2', 'a1f2', 
                'a1a3', 'a1b3', 'a1c3', 'a1d3', 'a1e3', 'a1f3', 'a1a4', 'a1b4', 'a1c4', 'a1d4', 'a1e4', 'a1f4', 
                'b1b1', 'b1c1', 'b1d1', 'b1e1', 'b1f1', 'b1a2', 'b1b2', 'b1c2', 'b1d2', 'b1e2', 'b1f2', 'b1a3', 
                'b1b3', 'b1c3', 'b1d3', 'b1e3', 'b1f3', 'b1a4', 'b1b4', 'b1c4', 'b1d4', 'b1e4', 'b1f4', 'c1c1', 
                'c1d1', 'c1e1', 'c1f1', 'c1a2', 'c1b2', 'c1c2', 'c1d2', 'c1e2', 'c1f2', 'c1a3', 'c1b3', 'c1c3', 
                'c1d3', 'c1e3', 'c1f3', 'c1a4', 'c1b4', 'c1c4', 'c1d4', 'c1e4', 'c1f4', 'd1d1', 'd1e1', 'd1f1', 
                'd1a2', 'd1b2', 'd1c2', 'd1d2', 'd1e2', 'd1f2', 'd1a3', 'd1b3', 'd1c3', 'd1d3', 'd1e3', 'd1f3',
                'd1a4', 'd1b4', 'd1c4', 'd1d4', 'd1e4', 'd1f4', 'e1e1', 'e1f1', 'e1a2', 'e1b2', 'e1c2', 'e1d2',
                'e1e2', 'e1f2', 'e1a3', 'e1b3', 'e1c3', 'e1d3', 'e1e3', 'e1f3', 'e1a4', 'e1b4', 'e1c4', 'e1d4',
                'e1e4', 'e1f4', 'f1f1', 'f1a2', 'f1b2', 'f1c2', 'f1d2', 'f1e2', 'f1f2', 'f1a3', 'f1b3', 'f1c3',
                'f1d3', 'f1e3', 'f1f3', 'f1a4', 'f1b4', 'f1c4', 'f1d4', 'f1e4', 'f1f4', 'a2a2', 'a2b2', 'a2c2',
                'a2d2', 'a2e2', 'a2f2', 'a2a3', 'a2b3', 'a2c3', 'a2d3', 'a2e3', 'a2f3', 'a2a4', 'a2b4', 'a2c4',
                'a2d4', 'a2e4', 'a2f4', 'b2b2', 'b2c2', 'b2d2', 'b2e2', 'b2f2', 'b2a3', 'b2b3', 'b2c3', 'b2d3', 
                'b2e3', 'b2f3', 'b2a4', 'b2b4', 'b2c4', 'b2d4', 'b2e4', 'b2f4', 'c2c2', 'c2d2', 'c2e2', 'c2f2', 
                'c2a3', 'c2b3', 'c2c3', 'c2d3', 'c2e3', 'c2f3', 'c2a4', 'c2b4', 'c2c4', 'c2d4', 'c2e4', 'c2f4', 
                'd2d2', 'd2e2', 'd2f2', 'd2a3', 'd2b3', 'd2c3', 'd2d3', 'd2e3', 'd2f3', 'd2a4', 'd2b4', 'd2c4', 
                'd2d4', 'd2e4', 'd2f4', 'e2e2', 'e2f2', 'e2a3', 'e2b3', 'e2c3', 'e2d3', 'e2e3', 'e2f3', 'e2a4', 
                'e2b4', 'e2c4', 'e2d4', 'e2e4', 'e2f4', 'f2f2', 'f2a3', 'f2b3', 'f2c3', 'f2d3', 'f2e3', 'f2f3', 
                'f2a4', 'f2b4', 'f2c4', 'f2d4', 'f2e4', 'f2f4', 'a3a3', 'a3b3', 'a3c3', 'a3d3', 'a3e3', 'a3f3', 
                'a3a4', 'a3b4', 'a3c4', 'a3d4', 'a3e4', 'a3f4', 'b3b3', 'b3c3', 'b3d3', 'b3e3', 'b3f3', 'b3a4', 
                'b3b4', 'b3c4', 'b3d4', 'b3e4', 'b3f4', 'c3c3', 'c3d3', 'c3e3', 'c3f3', 'c3a4', 'c3b4', 'c3c4', 
                'c3d4', 'c3e4', 'c3f4', 'd3d3', 'd3e3', 'd3f3', 'd3a4', 'd3b4', 'd3c4', 'd3d4', 'd3e4', 'd3f4', 
                'e3e3', 'e3f3', 'e3a4', 'e3b4', 'e3c4', 'e3d4', 'e3e4', 'e3f4', 'f3f3', 'f3a4', 'f3b4', 'f3c4', 
                'f3d4', 'f3e4', 'f3f4', 'a4a4', 'a4b4', 'a4c4', 'a4d4', 'a4e4', 'a4f4', 'b4b4', 'b4c4', 'b4d4', 
                'b4e4', 'b4f4', 'c4c4', 'c4d4', 'c4e4', 'c4f4', 'd4d4', 'd4e4', 'd4f4', 'e4e4', 'e4f4', 'f4f4']


list_mat = [(a1,a1), (a1,b1), (a1,c1), (a1,d1), (a1,e1), (a1,f1), (a1,a2), (a1,b2), (a1,c2), (a1,d2), (a1,e2), (a1,f2),
            (a1,a3), (a1,b3), (a1,c3), (a1,d3), (a1,e3), (a1,f3), (a1,a4), (a1,b4), (a1,c4), (a1,d4), (a1,e4), (a1,f4), 
            (b1,b1), (b1,c1), (b1,d1), (b1,e1), (b1,f1), (b1,a2), (b1,b2), (b1,c2), (b1,d2), (b1,e2), (b1,f2), (b1,a3), 
            (b1,b3), (b1,c3), (b1,d3), (b1,e3), (b1,f3), (b1,a4), (b1,b4), (b1,c4), (b1,d4), (b1,e4), (b1,f4), (c1,c1), 
            (c1,d1), (c1,e1), (c1,f1), (c1,a2), (c1,b2), (c1,c2), (c1,d2), (c1,e2), (c1,f2), (c1,a3), (c1,b3), (c1,c3), 
            (c1,d3), (c1,e3), (c1,f3), (c1,a4), (c1,b4), (c1,c4), (c1,d4), (c1,e4), (c1,f4), (d1,d1), (d1,e1), (d1,f1), 
            (d1,a2), (d1,b2), (d1,c2), (d1,d2), (d1,e2), (d1,f2), (d1,a3), (d1,b3), (d1,c3), (d1,d3), (d1,e3), (d1,f3), 
            (d1,a4), (d1,b4), (d1,c4), (d1,d4), (d1,e4), (d1,f4), (e1,e1), (e1,f1), (e1,a2), (e1,b2), (e1,c2), (e1,d2), 
            (e1,e2), (e1,f2), (e1,a3), (e1,b3), (e1,c3), (e1,d3), (e1,e3), (e1,f3), (e1,a4), (e1,b4), (e1,c4), (e1,d4), 
            (e1,e4), (e1,f4), (f1,f1), (f1,a2), (f1,b2), (f1,c2), (f1,d2), (f1,e2), (f1,f2), (f1,a3), (f1,b3), (f1,c3),
            (f1,d3), (f1,e3), (f1,f3), (f1,a4), (f1,b4), (f1,c4), (f1,d4), (f1,e4), (f1,f4), (a2,a2), (a2,b2), (a2,c2), 
            (a2,d2), (a2,e2), (a2,f2), (a2,a3), (a2,b3), (a2,c3), (a2,d3), (a2,e3), (a2,f3), (a2,a4), (a2,b4), (a2,c4), 
            (a2,d4), (a2,e4), (a2,f4), (b2,b2), (b2,c2), (b2,d2), (b2,e2), (b2,f2), (b2,a3), (b2,b3), (b2,c3), (b2,d3),
            (b2,e3), (b2,f3), (b2,a4), (b2,b4), (b2,c4), (b2,d4), (b2,e4), (b2,f4), (c2,c2), (c2,d2), (c2,e2), (c2,f2), 
            (c2,a3), (c2,b3), (c2,c3), (c2,d3), (c2,e3), (c2,f3), (c2,a4), (c2,b4), (c2,c4), (c2,d4), (c2,e4), (c2,f4), 
            (d2,d2), (d2,e2), (d2,f2), (d2,a3), (d2,b3), (d2,c3), (d2,d3), (d2,e3), (d2,f3), (d2,a4), (d2,b4), (d2,c4), 
            (d2,d4), (d2,e4), (d2,f4), (e2,e2), (e2,f2), (e2,a3), (e2,b3), (e2,c3), (e2,d3), (e2,e3), (e2,f3), (e2,a4), 
            (e2,b4), (e2,c4), (e2,d4), (e2,e4), (e2,f4), (f2,f2), (f2,a3), (f2,b3), (f2,c3), (f2,d3), (f2,e3), (f2,f3), 
            (f2,a4), (f2,b4), (f2,c4), (f2,d4), (f2,e4), (f2,f4), (a3,a3), (a3,b3), (a3,c3), (a3,d3), (a3,e3), (a3,f3), 
            (a3,a4), (a3,b4), (a3,c4), (a3,d4), (a3,e4), (a3,f4), (b3,b3), (b3,c3), (b3,d3), (b3,e3), (b3,f3), (b3,a4), 
            (b3,b4), (b3,c4), (b3,d4), (b3,e4), (b3,f4), (c3,c3), (c3,d3), (c3,e3), (c3,f3), (c3,a4), (c3,b4), (c3,c4), 
            (c3,d4), (c3,e4), (c3,f4), (d3,d3), (d3,e3), (d3,f3), (d3,a4), (d3,b4), (d3,c4), (d3,d4), (d3,e4), (d3,f4), 
            (e3,e3), (e3,f3), (e3,a4), (e3,b4), (e3,c4), (e3,d4), (e3,e4), (e3,f4), (f3,f3), (f3,a4), (f3,b4), (f3,c4), 
            (f3,d4), (f3,e4), (f3,f4), (a4,a4), (a4,b4), (a4,c4), (a4,d4), (a4,e4), (a4,f4), (b4,b4), (b4,c4), (b4,d4), 
            (b4,e4), (b4,f4), (c4,c4), (c4,d4), (c4,e4), (c4,f4), (d4,d4), (d4,e4), (d4,f4), (e4,e4), (e4,f4), (f4,f4)]


    

In [5]:
def fromSymtoCode(expression):
    """
    This code transforms the symbolic writing into strings of code.
    """
    if expression == 0:
        return expression
    else:
        diction =  {Bra(g*g)*Bra(g*g).dual:"gggg",
                    Bra(u*u)*Bra(u*u).dual:"uuuu",
                    Bra(g*g)*Bra(j*j).dual:"ggjj",
                    Bra(g*j)*Bra(g*j).dual:"gjjg",
                    Bra(u*u)*Bra(v*v).dual:"uuvv",
                    Bra(u*v)*Bra(u*v).dual:"uvvu",
                    Bra(g*g)*Bra(u*u).dual:"gguu",
                    Bra(g*u)*Bra(u*g).dual:"guug",
                    Bra(g*g)*Bra(v*v).dual:"ggvv",
                    Bra(g*v)*Bra(g*v).dual:"gvvg",
                    Bra(g*u)*Bra(j*v).dual:"gujv",
                    Bra(g*v)*Bra(j*u).dual:"gvju",
                    Bra(g*j)*Bra(u*v).dual:"gjuv",
                    Bra(g*g)*Bra(g*j).dual:"gggj",
                    Bra(g*j)*Bra(u*u).dual:"gjuu",
                    Bra(g*u)*Bra(u*j).dual:"guuj",
                    Bra(u*u)*Bra(u*v).dual:"uuuv",
                    Bra(g*g)*Bra(u*v).dual:"gguv",
                    Bra(g*u)*Bra(v*g).dual:"guvg"}
        # Here the terms in expression are subsituted by the one defined in new_expr
        new_expr = expression.subs(diction)
        diction2 = {g**2:"g_h_g",
                    u**2:"u_h_u",
                    g*j:"g_h_j",
                    u*v:"u_h_v"}
        new_expr2 = new_expr.subs(diction2)
  
    return new_expr2

## This part produces the 300 matrix elements (Hermitian matrices) and evaluates their mono- and bi-electronic integrals
Then, it transforms everything into a code that can be used for the evaluation of matrices.

In [6]:
for i in range(len(list_mat)):
    A,B = list_mat[i]
    elements = fromSymtoCode(minor_det_total(A,B))        
    print(list_name_mat[i] + " = " + str(elements))

a1a1 = 4*S1**2*gjjg - 8*S1*gggj + 4*g_h_g*(1 - S1**2) + 4*g_h_j*(S1**3 - S1) + 2*gggg + 2*ggjj + 2*(1 - S1**2)*(ggjj - gjjg)
a1b1 = S1**2*gvvg - 2*S1*guuj + guug
a1c1 = 0
a1d1 = 0
a1e1 = 0
a1f1 = S1**2*gujv - 2*S1*guvg + gvju
a1a2 = S1**2*gvvg - 2*S1*guuj + guug
a1b2 = 0
a1c2 = 0
a1d2 = 0
a1e2 = S1**2*gujv - 2*S1*guvg + gvju
a1f2 = 0
a1a3 = 0
a1b3 = 0
a1c3 = (1 - S1**2)*(gujv - gvju)
a1d3 = S1**2*gvju - 2*S1*guvg + gujv
a1e3 = 0
a1f3 = 0
a1a4 = 0
a1b4 = 0
a1c4 = S1**2*gvju - 2*S1*guvg + gujv
a1d4 = (1 - S1**2)*(gujv - gvju)
a1e4 = 0
a1f4 = 0
b1b1 = 2*g_h_g + gggg + 4*ggvv - 2*gvvg + 2*u_h_u + uuuu
b1c1 = 0
b1d1 = 0
b1e1 = 0
b1f1 = -S1*S3*g_h_g - S1*S3*ggvv - S1*S3*u_h_u - S1*u_h_v - S1*uuuv + S1*(-gguv + guvg) - S3*g_h_j - S3*gggj + S3*(-gjuu + guuj) - gjuv
b1a2 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j + 2*S1*S3*gjuv - 2*S1*S3*(-gjuv + gujv) + S3**2*gjjg
b1b2 = S3**2*gvvg - 2*S3*guvg + guug
b1c2 = 0
b1d2 = 0
b1e2 = -S1*S3*g_h_g - S1*S3*ggvv - S1*S3*u_h_u - S1*u_h_v - S1*uuuv 

d3a4 = 0
d3b4 = 0
d3c4 = 2*S1**2*S3*u_h_v + S1**2*uvvu + 2*S1*S3**2*g_h_j + 2*S1*S3*gjuv - 2*S1*S3*(-gjuv + gujv) + S3**2*gjjg
d3d4 = S1*S3*gvju - S1*guuj - S3*guvg + guug
d3e4 = 0
d3f4 = 0
e3e3 = -2*S1*g_h_j - 2*S1*gjuu + 2*S1*(-gjuu + guuj) + 2*g_h_g + ggjj + 2*gguu + 2*ggvv - gjjg - guug - gvvg + 2*u_h_u*(1 - S1**2) + uuvv*(1 - S1**2)
e3f3 = 0
e3a4 = 0
e3b4 = 0
e3c4 = 0
e3d4 = 0
e3e4 = S1*S3*gujv - S1*guuj - S3*guvg + guug
e3f4 = 0
f3f3 = -2*S3*gguv - 2*S3*u_h_v - 2*S3*(gguv - guvg) + 2*g_h_g*(1 - S3**2) + ggjj*(1 - S3**2) + 2*gguu + 2*ggvv - guug - gvvg + 2*u_h_u + uuvv - uvvu
f3a4 = 0
f3b4 = 0
f3c4 = 0
f3d4 = 0
f3e4 = 0
f3f4 = S1*S3*gujv - S1*guuj - S3*guvg + guug
a4a4 = -2*S1*g_h_j - 2*S1*gggj - 2*S1*gjuu + g_h_g*(1 - S1**2) + 2*g_h_g + gggg + 2*ggjj + gguu + ggvv - gjjg + u_h_u*(1 - S1**2) + (1 - S1**2)*(ggvv - gvvg)
a4b4 = S1*S3*gvju - S1*guuj - S3*guvg + guug
a4c4 = 0
a4d4 = 0
a4e4 = 0
a4f4 = 0
b4b4 = -2*S3*gguv - 2*S3*u_h_v - 2*S3*uuuv + g_h_g*(1 - S3**2) + gguu + ggvv + u_h_